In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016,2024))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep=5, retries=6):
    html = None
    for i in range(1, retries+1):
        time.sleep(i * sleep)
        
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue 
        else:
            break
            
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2021_games.html
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [7]:
standing_files = os.listdir(STANDINGS_DIR)

In [8]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscores" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")

        if not html:
            continue

        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
standing_files = [s for s in standing_files if ".html" in s]

In [10]:
for f in standing_files:
    file_path = os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(file_path)

Bulls vs Nets, November 1, 2022 | Basketball-Reference.com
Warriors vs Heat, November 1, 2022 | Basketball-Reference.com
Magic vs Thunder, November 1, 2022 | Basketball-Reference.com
Timberwolves vs Suns, November 1, 2022 | Basketball-Reference.com
Wizards vs 76ers, November 2, 2022 | Basketball-Reference.com
Celtics vs Cavaliers, November 2, 2022 | Basketball-Reference.com
Kings vs Heat, November 2, 2022 | Basketball-Reference.com
Hawks vs Knicks, November 2, 2022 | Basketball-Reference.com
Hornets vs Bulls, November 2, 2022 | Basketball-Reference.com
Clippers vs Rockets, November 2, 2022 | Basketball-Reference.com
Pistons vs Bucks, November 2, 2022 | Basketball-Reference.com
Raptors vs Spurs, November 2, 2022 | Basketball-Reference.com
Jazz vs Mavericks, November 2, 2022 | Basketball-Reference.com
Grizzlies vs Trail Blazers, November 2, 2022 | Basketball-Reference.com
Pelicans vs Lakers, November 2, 2022 | Basketball-Reference.com
Warriors vs Magic, November 3, 2022 | Basketball-Refe